## Intermediate demo notebook

First we import the libraries we need

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt

# Third-party imports
import camb

# Project imports
import pyhalomodel as halo

Now we set the cosmological parameters. If `sigma_8_set = True` we scale the linear power spectrum to account for the new `sigma_8` value.  

In [ ]:
# Set cosmological parameters
Omega_c = 0.25
Omega_b = 0.05
Omega_k = 0.0
h = 0.7
As = 2e-9
ns = 0.96
w = -1.0
wa = 0.0
m_nu = 0.0 # [eV]
sigma_8_set = True # If True uses the following value
sigma_8  = 0.8

Set some plot parameters

In [ ]:
# Plot colours
col_lin = 'grey'
col_mat = 'C0'
col_gal = 'C1'
col_cen = 'C3'
col_sat = 'C0'
col_mg  = 'C4'
col_cs  = 'C5'

# Plot line styles
ls_li = '-'
ls_hm = '-'
ls_2h = '--'
ls_1h = ':'

# Plot labels
klab = r'$k\,/\,h \mathrm{Mpc}^{-1}$'
Pklab = r'$P(k)/(h^{-1}\,\mathrm{Mpc})^3$'

Now let's set some parameters that we will use throughout the calculation: First, we set a range of wavenumbers, `k`, and then fill array, `ks`, we then set a redshift, `z`.

In [ ]:
# Wavenumber range [h/Mpc]
kmin, kmax = 1e-3, 10.
nk = 101
ks = np.logspace(np.log10(kmin), np.log10(kmax), nk)

# Redshift
z = 0.

# CAMB
zmax_CAMB = 2.
kmax_CAMB = 200.

Now we initialise `CAMB` and produce a linear matter power spectrum. We have the option here to scale the power by an input `sigma_8` value.

In [ ]:
# Sets cosmological parameters in camb to calculate the linear power spectrum
pars = camb.CAMBparams()
wb = Omega_b*h**2
wc = Omega_c*h**2

# This function sets standard and helium set using BBN consistency
pars.set_cosmology(ombh2=wb, omch2=wc, H0=100.*h, mnu=m_nu, omk=Omega_k)
pars.set_dark_energy(w=w, wa=wa, dark_energy_model='ppf') 
pars.InitPower.set_params(As=As, ns=ns, r=0.)
pars.set_matter_power(redshifts=[z], kmax=kmax_CAMB) # Setup the linear matter power spectrum

# Scale 'As' to be correct for the desired 'sigma_8' value if necessary
if sigma_8_set:
    camb_results = camb.get_results(pars)
    sigma_8_init = (camb_results.get_sigma8()[[z].index(0.)]).item()
    print('Initial sigma_8:', sigma_8_init)
    scaling = (sigma_8/sigma_8_init)**2
    As *= scaling
    pars.InitPower.set_params(As=As, ns=ns, r=0.)

# Now get the linear power spectrum
Pk_lin = camb.get_matter_power_interpolator(pars, 
                                            nonlinear=False, 
                                            hubble_units=True, 
                                            k_hunit=True, 
                                            kmax=kmax_CAMB,
                                            var1=camb.model.Transfer_tot,
                                            var2=camb.model.Transfer_tot, 
                                            zmax=zmax_CAMB,
                                           )
Omega_m  = pars.omegam # Also extract the matter density
Pks_lin = Pk_lin.P(z, ks) # Single out the linear P(k) interpolator and evaluate linear power
camb_results = camb.get_results(pars)
sigma_8 = (camb_results.get_sigma8()[[z].index(0.)]).item()
if sigma_8_set: print('Final sigma_8:', sigma_8)

Now we can create a halo model, which needs to be reinitialised at each different redshift. In this example notebook we are only doing calculations at a single redshift, so we initialise the halo model with that in mind. We also need to choose a mass function and linear halo bias, in this example these come from `Tinker et al. (2010)`, and we need to choose a halo definition, here we choose $\Delta_\mathrm{v}=330$, so haloes are defined to be spherical objects that contain an average density that is $330$ times greater than the mean background universe, which is approximately the virial (spherical-collapse) definition for this cosmology at $z=0$.

In [ ]:
# Initialise halo model
hmod = halo.model(z, Omega_m, name='Tinker et al. (2010)', Dv=330.)
print(hmod)

Now we set a range of halo masses, `M`, and fill an array, `Ms`. This halo-mass range needs to be wide enough that it includes all 'interesting' halo masses from the point of view of the calculation, and the mass-spacing needs to be fine enough that the calculation converges; in an actual use-case the effect on power spectra of both mass range and spacing should be convergence tested. Then we find the Lagrangian radii, `Rs`, corresponding the the halo masses.

In [ ]:
# Halo mass range [Msun/h] over which to integrate
Mmin, Mmax = 1e9, 1e17
nM = 256
Ms = np.logspace(np.log10(Mmin), np.log10(Mmax), nM)

# Lagrangian radii [Mpc/h] corresponding to halo masses
Rs = hmod.Lagrangian_radius(Ms)

Next, we need to get an array of `sigma(R)` values. This requires an integral over the linear power spectrum times the Fourier transform of a top hat function, which is the oscillatory sinc function. We can get this from `CAMB`.

In [ ]:
# Get sigma(R) from CAMB
sigmaRs = camb_results.get_sigmaR(Rs, hubble_units=True, return_R_z=False)[[z].index(z)]

First, we will use the halo model to compute the matter power spectrum, to do this we need to define a matter halo. We use the `haloprof` class, which contains the Fourier window function of the halo profile `W(M,k)` that is necessary for the power-spectrum calculation. This is a 2D array of the window function evaluated at `(ks, Ms)` values defined above, so if these change then the `haloprof` must be updated. We break the function $W(M,k)=N(M)U(M,k)/{\rm norm}$ where $W(M,k)$ has dimensions of field multiplied by volume, $N(M)/{\rm norm}$ has dimension of field multiplied by volume and $U(M,k)$ is dimensionless and should tend to unity as k tends to zero, because its real space counter part is normalised. 

In our case, we are going to compute the power spectrum of matter overdensity (density contrast: $\delta_{\rm m}=(\rho-\bar\rho)/\bar\rho$), which is dimensionless. We take $N(M)=M$, ${\rm norm}=\bar\rho$, such that $N(M)/{\rm norm}$ = $M/\bar\rho$ and has dimensions of volume. We could also have set $N(M)=M/\bar\rho$ and ${\rm norm}=1$ and would get the same results. The distinction between these two approaches is important for discrete tracers, but not otherwise.

We also need to set the optional `mass_tracer=True` in the halo profile, to let it know that our profile corresponds to a matter profile. This is important because the contribution to the matter power of haloes below `Mmin`, set above, is important to the calculation (much of the mass in the Universe is in very low mass haloes), and this flag allows this to be taken into account in a consistent way.

In [ ]:
# Halo window functions
rvs = hmod.virial_radius(Ms)
cs = halo.concentration(Ms, z, method='Duffy et al. (2008)', halo_definition='Mvir') # Specify 'Mvir' for Duffy c(M)
Uk = halo.window_function(ks, rvs, cs, profile='NFW')

# Initialise profile class
matter_profile = halo.profile.Fourier(ks, Ms, Uk, amplitude=Ms, normalisation=hmod.rhom, mass_tracer=True) # Need mass_tracer=True here
print(matter_profile)

Now we are in a position to combine our halo model with our matter profile and do a calculation of the power spectrum. Note that profile must be provided as a dictionary: `{'m': matter_profile}` rather than `matter_profile`, where the key `'m'` can be chosen by the user and identifies that the `matter_profile` corresponds to field `'m'`. The reason for this will become apparent soon.

In [ ]:
# Calculate the halo-model power spectrum
Pk_2h, Pk_1h, Pk_hm = hmod.power_spectrum(ks, Pks_lin, Ms, sigmaRs, {'m': matter_profile})

Now let's do some plotting. Note carefully that the output arrays `Pk_2h`, `Pk_1h`, and `Pk_hm` are three dimensional, with `k` in the third dimension. In this first example the first two dimensions are length `1` because we have only provided one profile, the `matter_profile`. So to access the power spectrum we need `Pk_xx['m-m']`.

In [ ]:
# Initialise plot
plt.subplots(3, 1, figsize=(6., 7.), dpi=100)
Pkmin, Pkmax = 1e1, 1e5
rmin, rmax = 1e-1, 1e2
smin, smax = 0.0, 1.1
kmin_plot, kmax_plot = 1e-3, 1e1

# P(k)
plt.subplot(3, 1, 1)
plt.loglog(ks, Pks_lin, color=col_lin, label='Linear')
plt.loglog(ks, Pk_2h['m-m'], color=col_mat, ls=ls_2h, label='Two-halo term')
plt.loglog(ks, Pk_1h['m-m'], color=col_mat, ls=ls_1h, label='One-halo term')
plt.loglog(ks, Pk_hm['m-m'], color=col_mat, ls=ls_hm, label='Matter')
plt.xticks([])
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_\mathrm{mm}(k)\,/\,(h^{-1} \mathrm{Mpc})^3$')
plt.ylim((Pkmin, Pkmax))
plt.legend(fontsize='9', loc='lower left')

# Residual with linear
plt.subplot(3, 1, 2)
plt.loglog(ks, Pks_lin/Pks_lin, color=col_lin)
plt.loglog(ks, Pk_2h['m-m']/Pks_lin, color=col_mat, ls=ls_2h)
plt.loglog(ks, Pk_1h['m-m']/Pks_lin, color=col_mat, ls=ls_1h)
plt.loglog(ks, Pk_hm['m-m']/Pks_lin, color=col_mat, ls=ls_hm)
plt.xticks([])
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_\mathrm{mm}(k)\,/\,P^\mathrm{lin}_{\rm mm}(k)$')
plt.ylim((rmin, rmax))

# Residual with halo-model matter power
plt.subplot(3, 1, 3)
plt.semilogx(ks, Pks_lin/Pk_hm['m-m'], color=col_lin, ls='-')
plt.semilogx(ks, Pk_2h['m-m']/Pk_hm['m-m'], color=col_mat, ls=ls_2h)
plt.semilogx(ks, Pk_1h['m-m']/Pk_hm['m-m'], color=col_mat, ls=ls_1h)
plt.semilogx(ks, Pk_hm['m-m']/Pk_hm['m-m'], color=col_mat, ls=ls_hm)
plt.xlabel(klab)
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_\mathrm{mm}(k)\,/\,P_\mathrm{mm}^\mathrm{hm}(k)$')
plt.ylim((smin, smax))

# Finish
plt.tight_layout()
plt.show()

The upper panel shows the halo-model matter power spectrum broken into two- and one-halo components, we see that the two-halo term dominates at large scales while the one-halo term dominates at small scales, as expected. We see that the one-halo term has a constant 'shot-noise' contribution at large scales, and we see that the two-halo term deviates from the linear spectrum only at small scales, and only where the one-halo term is dominating the overall spectrum. The shot-noise contribution of the one-halo term at large scales is unphysical, and arises because neither mass nor momentum conservation have been imposed in the initial step of the derivation of the halo model, but this has very little impact in practice, although the one-halo term will eventually (and unphysically) dominate the power at ultra-large scales. The middle panel shows each spectra relative to linear and the bottom shows the contribution of the two- and one-halo terms to the total. Note that each term contributes equally at $k\sim 0.5 h\mathrm{Mpc}^{-1}$, which is the transition region at $z=0$. The slight (and unphysical) deviation of the total power from the linear result can be seen at $k\sim 10^{-3} h\mathrm{Mpc}^{-1}$.

Now let's calculate some galaxy spectra, first we need to specify a halo-occupation distribution (HOD) model to assign galaxies to haloes. In this example we choose a simple HOD, one that illustrates the basic point.

In [ ]:
# Simple HOD model
def HOD(M, Mmin):
    return np.rint(M/Mmin)
Ng = HOD(Ms, Mmin=1e11)

# Compute the mean galaxy density corresponding to our HOD
rhog = hmod.average(Ms, sigmaRs, Ng)
print('Mean galaxy density [(Mpc/h)^-3]:', rhog)

# Plot HOD
plt.loglog(Ms, Ng, color=col_gal)
plt.xlabel('Halo mass [$h^{-1}\,M_\odot$]')
plt.xlim(right=1e16)
plt.ylabel('Number of galaxies')
plt.ylim(bottom=1e-1)
plt.show()

The unusual HOD has step features, but that is no problem for `halomodel`.

Now we can create a galaxy profile by combining the HOD with a halo window. Here we choose an isothermal profile `rho(r) ~ 1/r^2` for the galaxies and we set the `discrete_tracer=True` flag to let the calculation know that the halo profile corresponds to that of a discrete tracer, which ensures that `<N(N-1)>` is used in the one-halo term, rather than `<N^2>`. In the `W(M,k)=N(M)W(M,k)/norm` language we used before we have `N(M) = Ng(M)`, the number of galaxies in each halo, and `norm=rhog` the overall number-density of galaxies (dimension 1/volume). For discrete tracers (only) it is important to make the distinction between `N` and `norm`, and we would get different results if we set `N(M)=Ng(M)/norm` and `norm=1.`. We can then do a power spectrum calculation.

In [ ]:
# Halo window function
rvs = hmod.virial_radius(Ms)
Uk = halo.window_function(ks, rvs, profile='isothermal')

# Galaxy profile: Need discrete_tracer=True here because profile is of a discrete galaxys
galaxy_profile = halo.profile.Fourier(ks, Ms, Uk, amplitude=Ng, normalisation=rhog, discrete_tracer=True)
print(galaxy_profile)

# Calculate the halo-model power spectrum
Pk_2h, Pk_1h, Pk_hm = hmod.power_spectrum(ks, Pks_lin, Ms, sigmaRs, {'g': galaxy_profile})

And we can plot the results once more...

In [ ]:
# Initialise plot
plt.subplots(3, 1, figsize=(6., 7.))
Pkmin = 1e1; Pkmax = 1e5
kmin_plot = 1e-3; kmax_plot = 1e1

# P(k)
plt.subplot(3, 1, 1)
plt.loglog(ks, Pks_lin, color=col_lin, label='Linear')
plt.loglog(ks, Pk_2h['g-g'], color=col_gal, ls='--', label='Two-halo term')
plt.loglog(ks, Pk_1h['g-g'], color=col_gal, ls=':', label='One-halo term')
plt.loglog(ks, Pk_hm['g-g'], color=col_gal, ls='-', label='Galaxies')
plt.xticks([])
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_\mathrm{gg}(k)\,/\,(h^{-1} \mathrm{Mpc})^3$')
plt.ylim((Pkmin, Pkmax))
plt.legend(fontsize='9')

# Residual with linear
plt.subplot(3, 1, 2)
plt.loglog(ks, Pks_lin/Pks_lin, color=col_lin)
plt.loglog(ks, Pk_2h['g-g']/Pks_lin, color=col_gal, ls='--')
plt.loglog(ks, Pk_1h['g-g']/Pks_lin, color=col_gal, ls=':')
plt.loglog(ks, Pk_hm['g-g']/Pks_lin, color=col_gal, ls='-')
plt.xticks([])
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_\mathrm{gg}(k)\,/\,P^\mathrm{lin}(k)$')

# Residual with halo-model matter power
plt.subplot(3, 1, 3)
plt.semilogx(ks, Pks_lin/Pk_hm['g-g'], color=col_lin, ls='-')
plt.semilogx(ks, Pk_2h['g-g']/Pk_hm['g-g'], color=col_gal, ls='--')
plt.semilogx(ks, Pk_1h['g-g']/Pk_hm['g-g'], color=col_gal, ls=':')
plt.semilogx(ks, Pk_hm['g-g']/Pk_hm['g-g'], color=col_gal, ls='-')
plt.xlabel(klab)
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_\mathrm{gg}(k)\,/\,P_\mathrm{gg}^\mathrm{hm}(k)$')
plt.ylim(bottom=0.)

# Finish
plt.tight_layout()
plt.show()

The galaxy spectrum looks superficially similar to the matter spectrum but the details are different. Note the large-scale bias means that the amplitude of the galaxy spectrum is elevated with respect to the linear spectrum. The relative amplitude of this offset is the squared bias of our galaxy sample, and so it looks like our HOD corresponds to a galaxy sample with $b \simeq 1.5$ (note that the power ratio goes like $b^2$). Note that in the case of the galaxy sample the large-scale shot noise in the one-halo term, as the galaxy distribution is not restrained by either mass or momentum conservation. However, in this example the classic $1/n$ galaxy shot noise has been subtracted from the spectrum, which is already handled by the one-halo calculation. If one would rather retain this shot noise, then one can set the `shot=True` flag in the `halo.Pk_hm` calculation.

In the previous example, the HOD we constructed had only integer numbers of galaxies assigned to each halo. In reality, the way galaxies occupy haloes is expected to be stochastic, with some variation about a mean occupation number for each halo mass. This variance will contribute to the power spectrum, and always adds a little bit of power. `halomodel` can include this. Let's look at the difference that this makes in the case that we assume a Poisson variance, where the variance is exactly equal to the mean occupation number of the halo.

In [ ]:
# Halo window function
def HOD(M, Mmin):
    return M/Mmin
Ng = HOD(Ms, Mmin=1e12)
rhog = hmod.average(Ms, sigmaRs, Ng)
rvs = hmod.virial_radius(Ms)
Uk = halo.window_function(ks, rvs, profile='isothermal')

# Galaxy profile: Need discrete_tracer=True here because profile is of a discrete galaxys
galaxy_profile_novar = halo.profile.Fourier(ks, Ms, Uk, amplitude=Ng, normalisation=rhog, discrete_tracer=True)
galaxy_profile_var = halo.profile.Fourier(ks, Ms, Uk, amplitude=Ng, normalisation=rhog, variance=Ng, discrete_tracer=True)

# Calculate the halo-model power spectrum
Pk_2h_novar, Pk_1h_novar, Pk_hm_novar = hmod.power_spectrum(ks, Pks_lin, Ms, sigmaRs, {'g': galaxy_profile_novar})
Pk_2h_var, Pk_1h_var, Pk_hm_var = hmod.power_spectrum(ks, Pks_lin, Ms, sigmaRs, {'g': galaxy_profile_var})

In [ ]:
# Axis limits
Pkmin = 1e1; Pkmax = 1e5
kmin_plot = 1e-3; kmax_plot = 1e1
col_var, col_novar = col_gal, col_gal
alpha = 0.3

# Initialise plot
plt.subplots(2, 1, figsize=(6., 6.))

# P(k)
plt.subplot(2, 1, 1)
for ls, label in zip([ls_2h, ls_1h, ls_hm], ['Two-halo term', 'One-halo term', 'Galaxies']):
    plt.plot(np.nan, color='black', ls=ls, label=label)
plt.loglog(ks, Pks_lin, color=col_lin, label='Linear')
for ls, label, Pk in zip([ls_2h, ls_1h, ls_hm], [None, None, 'No variance'], [Pk_2h_novar['g-g'], Pk_1h_novar['g-g'], Pk_hm_novar['g-g']]):
    plt.loglog(ks, Pk, color=col_novar, ls=ls, label=label, alpha=alpha)
for ls, label, Pk in zip([ls_2h, ls_1h, ls_hm], [None, None, 'With variance'], [Pk_2h_var['g-g'], Pk_1h_var['g-g'], Pk_hm_var['g-g']]):
    plt.loglog(ks, Pk, color=col_var, ls=ls, label=label)
plt.xticks([])
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_\mathrm{gg}(k)\,/\,(h^{-1} \mathrm{Mpc})^3$')
plt.ylim((Pkmin, Pkmax))
plt.legend(fontsize='9', ncol=2)

# Residual with halo-model matter power
plt.subplot(2, 1, 2)
plt.semilogx(ks, Pks_lin/Pk_hm_var['g-g'], color=col_lin, ls='-')
for ls, Pk in zip([ls_2h, ls_1h, ls_hm], [Pk_2h_novar['g-g'], Pk_1h_novar['g-g'], Pk_hm_novar['g-g']]):
    plt.semilogx(ks, Pk/Pk_hm_var['g-g'], color=col_novar, ls=ls, alpha=alpha)
for ls, Pk in zip([ls_2h, ls_1h, ls_hm], [Pk_2h_var['g-g'], Pk_1h_var['g-g'], Pk_hm_var['g-g']]):
    plt.semilogx(ks, Pk/Pk_hm_var['g-g'], color=col_var, ls=ls)
plt.xlabel(klab)
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_\mathrm{gg}(k)\,/\,P_\mathrm{gg}^\mathrm{hm}(k)$')
plt.ylim(bottom=0.)

# Finish
plt.tight_layout()
plt.show()

We see that the variance adds power in the one-halo term, with the result that the power at $k=10h\mathrm{Mpc}^{-1}$ is almost twice that if the variance is neglected, and there are $5\%$-level effects at $k\simeq 1h\mathrm{Mpc}^{-1}$ The variance is important, and should be considered in general.

We can also decide whether or not to include the galaxy shot-noise in our predictions. In some conventions this is subtracted from the power, but not always. 

In [ ]:
# Halo profiles
Ng = HOD(Ms, Mmin=1e12)
rhog = hmod.average(Ms, sigmaRs, Ng)
rvs = hmod.virial_radius(Ms)
Uk = halo.window_function(ks, rvs, profile='isothermal')
galaxy_profile = halo.profile.Fourier(ks, Ms, Uk, amplitude=Ng, normalisation=rhog, variance=Ng, discrete_tracer=True)

# Calculate the halo-model power spectrum
Pk_2h_noshot, Pk_1h_noshot, Pk_hm_noshot = hmod.power_spectrum(ks, Pks_lin, Ms, sigmaRs, {'g': galaxy_profile}, subtract_shotnoise=True)
Pk_2h_shot, Pk_1h_shot, Pk_hm_shot = hmod.power_spectrum(ks, Pks_lin, Ms, sigmaRs, {'g': galaxy_profile}, subtract_shotnoise=False)

In [ ]:
# Initialise plot
plt.subplots(2, 1, figsize=(6., 6.))
Pkmin = 1e1; Pkmax = 1e5
kmin_plot = 1e-3; kmax_plot = 1e1
alpha = 0.3

# P(k)
plt.subplot(2, 1, 1)
for ls, label in zip([ls_2h, ls_1h, ls_hm], ['Two-halo term', 'One-halo term', 'Galaxies']):
    plt.plot(np.nan, color='black', ls=ls, label=label)
plt.loglog(ks, Pks_lin, color=col_lin, label='Linear')
for ls, label, Pk in zip([ls_2h, ls_1h, ls_hm], [None, None, 'Shot noise subtracted'], [Pk_2h_noshot['g-g'], Pk_1h_noshot['g-g'], Pk_hm_noshot['g-g']]):
    plt.loglog(ks, Pk, color=col_gal, ls=ls, label=label)
for ls, label, Pk in zip([ls_2h, ls_1h, ls_hm], [None, None, 'Shot noise present'], [Pk_2h_shot['g-g'], Pk_1h_shot['g-g'], Pk_hm_shot['g-g']]):
    plt.loglog(ks, Pk, color=col_gal, ls=ls, label=label, alpha=alpha)
plt.xticks([])
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_\mathrm{gg}(k)\,/\,(h^{-1} \mathrm{Mpc})^3$')
plt.ylim((Pkmin, Pkmax))
plt.legend(fontsize='9', ncol=2)

# Residual with halo-model matter power
plt.subplot(2, 1, 2)
plt.semilogx(ks, Pks_lin/Pk_hm_noshot['g-g'], color=col_lin, ls='-')
for ls, Pk in zip([ls_2h, ls_1h, ls_hm], [Pk_2h_novar['g-g'], Pk_1h_noshot['g-g'], Pk_hm_noshot['g-g']]):
    plt.semilogx(ks, Pk/Pk_hm_noshot['g-g'], color=col_gal, ls=ls)
for ls, Pk in zip([ls_2h, ls_1h, ls_hm], [Pk_2h_shot['g-g'], Pk_1h_shot['g-g'], Pk_hm_shot['g-g']]):
    plt.semilogx(ks, Pk/Pk_hm_noshot['g-g'], color=col_gal, ls=ls, alpha=alpha)
plt.xlabel(klab)
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_\mathrm{gg}(k)\,/\,P_\mathrm{no-shot-noise}^\mathrm{hm}(k)$')
plt.ylim(bottom=0.)

# Finish
plt.tight_layout()
plt.show()

We see that including shot noise adds significant power at small scales. The shot noise is assumed to be independent of scale and constant $1/n_\mathrm{g}$.

Now that we have created both matter and galaxy profiles, it is easy to cross correlate them by supplying a list of the two halo profiles. When we do this, the code calculates both auto spectra and also the cross spectrum. If we supply `n` halo profiles then we get the triangle number of `n` independent auto/cross spectra computed. These are stored and accessed via the keys of `Pk_xx`. For example, if we have `profiles = {'u': profile_u, 'v': profile_v}` then `Pk_xx['u-u']` will store the auto spectrum corresponding to `profile_u`, `Pk_xx['v-v']` will store the same but for `profile_v`, the cross spectra can be accessed via either of `Pk_xx['u-v']` or  `Pk_xx['v-u']`, so there is some redundancy but the cross calculations are performed only once.

In [ ]:
# Calculate the halo-model power spectrum
profiles = {'m': matter_profile, 'g': galaxy_profile}
Pk_2h, Pk_1h, Pk_hm = hmod.power_spectrum(ks, Pks_lin, Ms, sigmaRs, profiles)

...and plot, noting that `Pk_xx['m-m']` singles out the matter-matter (auto) spectrum, `Pk_xx['g-g']` singles out the galaxy-galaxy (auto) spectrum, and `Pk_xx['m-g']` singles out the matter-galaxy cross spectrum, and is identical to `Pk_xx['g-m']`:

In [ ]:
# Initialise plot
plt.subplots(3, 1, figsize=(6., 7.))
#plt.subplots_adjust(wspace=0.1, hspace=0.1) 
Pkmin = 1e1; Pkmax = 1e5
kmin_plot = 1e-3; kmax_plot = 1e1
rmin = 1e-1; rmax = 3e2
smin = 0.; smax = 4.

# Lists for plots
Pks = [Pk_2h['m-m'], Pk_1h['m-m'], Pk_hm['m-m'], Pk_hm['g-g'], Pk_hm['m-g']]
cols = 3*[col_mat]+[col_gal, col_mg]
lss = [ls_2h, ls_1h, ls_hm, ls_hm, ls_hm]
labs = [None, None, 'Matter', 'Galaxy', 'Matter-galaxy']

# P(k)
plt.subplot(3, 1, 1)
plt.loglog(ks, Pks_lin, color=col_lin, label='Linear')
for (ls, lab) in zip([ls_2h, ls_1h], ['Two-halo term', 'One-halo term']):
    plt.plot(np.nan, ls=ls, label=lab, color='black')
for (Pk, col, ls, lab) in zip(Pks, cols, lss, labs):
    plt.loglog(ks, Pk, color=col, ls=ls, label=lab)
plt.xticks([])
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_{\rm uv}(k)\,/\,(h^{-1} \mathrm{Mpc})^3$')
plt.ylim((Pkmin, Pkmax))
plt.legend(ncol=2, loc='lower left', fontsize='9')

# Lists for plots
Pks = [Pk_2h['m-m'], Pk_1h['m-m'], Pk_hm['m-m'],
       Pk_2h['g-g'], Pk_1h['g-g'], Pk_hm['g-g'],
       Pk_2h['m-g'], Pk_1h['m-g'], Pk_hm['m-g']]
cols = 3*[col_mat]+3*[col_gal]+3*[col_mg]
lss = 3*[ls_2h, ls_1h, ls_hm]

# Residual with linear
plt.subplot(3, 1, 2)
plt.loglog(ks, Pks_lin/Pks_lin, color=col_lin)
for (Pk, col, ls) in zip(Pks, cols, lss):
    plt.loglog(ks, Pk/Pks_lin, color=col, ls=ls)
plt.xticks([])
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_{\rm uv}(k)\,/\,P^\mathrm{lin}(k)$')
plt.ylim((rmin, rmax))

# Residual with halo-model matter power
plt.subplot(3, 1, 3)
plt.semilogx(ks, Pks_lin/Pk_hm['m-m'], color=col_lin)
for (Pk, col, ls) in zip(Pks, cols, lss):
    plt.semilogx(ks, Pk/Pk_hm['m-m'], color=col, ls=ls)
plt.xlabel(klab)
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_{\rm uv}(k)\,/\,P_\mathrm{mm}(k)$')
plt.ylim((smin, smax))

# Finish
plt.show()

We see that all spectra have the same shape at large scales, but that those spectra that involve galaxies are offset in amplitude. This tells us that the galaxy sample is positively biased, with $b\simeq1.5$. At smaller scales the spectra are all similarly shaped, but the exact scale dependence is different for each. This arises because of the different way that matter and galaxies occupy haloes.

Now, let's create a more realistic HOD with both central and satellite galaxies with their own unique statistical properties. We will use the Zheng et al. (2005) HOD model. In the more realistic example we also need to consider the variance both central and satellite occupation numbers at each halo mass. As is common, we will assume that central galaxies are Bernoulli distributed (either 0 or 1 with some probability) and that satellite galaxies are Poisson distributed.

In [ ]:
# Zheng et al. (2005) HOD with some reasonable parameters
N_cen, N_sat = halo.HOD_mean(Ms, method='Zheng et al. (2005)', Mmin=1e12, sigma=0.15, M0=1e12, M1=1e13, alpha=1.)
V_cen, V_sat, _ = halo.HOD_variance(N_cen, N_sat, central_condition=False)

# Compute the mean galaxy density corresponding to our HOD
rho_cen = hmod.average(Ms, sigmaRs, N_cen)
rho_sat = hmod.average(Ms, sigmaRs, N_sat)
rho_gal = rho_cen+rho_sat
print('Mean central galaxy density [(Mpc/h)^-3]:', rho_cen)
print('Mean satellite galaxy density [(Mpc/h)^-3]:', rho_sat)
print('Mean galaxy density [(Mpc/h)^-3]:', rho_gal)
print('Sample central galaxy fraction:', rho_cen/rho_gal)
print('Sample satellite galaxy fraction:', rho_sat/rho_gal)

# Plot HOD
plt.fill_between(Ms, N_cen+np.sqrt(V_cen), N_cen-np.sqrt(V_cen), color=col_cen, alpha=0.2)
plt.fill_between(Ms, N_sat+np.sqrt(V_sat), N_sat-np.sqrt(V_sat), color=col_sat, alpha=0.2)
plt.loglog(Ms, N_cen, color=col_cen, label='Central galaxies')
plt.loglog(Ms, N_sat, color=col_sat, label='Satellite galaxies')
plt.xlabel('Halo mass [$h^{-1}\,M_\odot$]')
plt.xlim((1e11, 1e15))
plt.ylabel('Number of galaxies')
plt.ylim((1e-1, 1e2))
plt.legend()
plt.show()

On plotting the HOD distribution as a function of halo mass we can see the mean occupancy and its variance both look quite reasonable.

Now we can set up the profiles of the central and satellite galaxies and compute the halo model power. Here we include the variance via `var` optional argument; note that this corresponds to the variance in `N(M)` at each `M`. We then sum the component power spectra to get the total. We need to sum central-central, central-satellite, satellite-central and satellite-satellite. Note that central-satellite is equal to satellite-central, but both must be included (or twice one of them). Also note that we define `norm=rho_gal` for each halo profile, rather than `norm=rho_cen` or `norm=rho_sat`, which allows us to sum them nicely at the end. Here we have assumed that the presence of central and satellite galaxies are entirely independent, which allows for the possibility of haloes hosting a satellite galaxy without hosting a central. With the particular parameters we have chosen this is not important, but in general one may wish to impose the 'central-condition' that satellite galaxies cannot be present without a central. This introduces a covariance between central and satellite occupation that cannot be included in the current version of `pyhalomodel`.

In [ ]:
# Halo window functions
rvs = hmod.virial_radius(Ms)
Uk_cen = halo.window_function(ks, rvs, profile='delta')
Uk_sat = halo.window_function(ks, rvs, profile='isothermal')

# Initialise profile class; need discrete_tracer=True here because profile is of a discrete tracer
central_profile = halo.profile.Fourier(ks, Ms, Uk_cen, amplitude=N_cen, normalisation=rho_gal, variance=V_cen, discrete_tracer=True)
satellite_profile = halo.profile.Fourier(ks, Ms, Uk_sat, amplitude=N_sat, normalisation=rho_gal, variance=V_sat, discrete_tracer=True)

# Calculate the halo-model power spectrum
profiles = {'c': central_profile, 's': satellite_profile}
_, _, Pk_hm = hmod.power_spectrum(ks, Pks_lin, Ms, sigmaRs, profiles)

# Sum to get the total galaxy power; cross terms are identical, but we need to include both in the sum
Pk_gg = Pk_hm['c-c']+Pk_hm['c-s']+Pk_hm['s-c']+Pk_hm['s-s']

... and plot the power spectra of centrals, satellites, their cross spectra and the total...

In [ ]:
# Initialise plot
plt.subplots(3, 1, figsize=(6., 7.))
Pkmin = 1e1; Pkmax = 1e5
rmin = 1e-1; rmax = 3e2
smin = 0.; smax = 1.1
kmin_plot = 1e-3; kmax_plot = 1e1

# Lists for plot
Pks = [Pk_hm['c-c'], Pk_hm['s-s'], 2.*Pk_hm['c-s'], Pk_gg]
cols = [col_cen, col_sat, col_cs, col_gal]
labs = ['Central galaxies', 'Satellite galaxies', 'Central-satellite', 'All galaxies']
lss = ['-', '-', '--', '-']

# P(k)
plt.subplot(3, 1, 1)
plt.loglog(ks, Pks_lin, color=col_lin, label='Linear')
for (Pk, col, lab, ls) in zip(Pks, cols, labs, lss):
    plt.loglog(ks, Pk, color=col, label=lab, ls=ls)
plt.xticks([])
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_{uv}(k)\,/\,(h^{-1} \mathrm{Mpc})^3$')
plt.ylim((Pkmin, Pkmax))
plt.legend(fontsize='9')

# Residual with linear
plt.subplot(3, 1, 2)
plt.loglog(ks, Pks_lin/Pks_lin, color=col_lin)
for (Pk, col, ls) in zip(Pks, cols, lss):
    plt.loglog(ks, Pk/Pks_lin, color=col, ls=ls)
plt.xticks([])
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_{uv}(k)\,/\,P^\mathrm{lin}(k)$')
plt.ylim((rmin, rmax))

# Residual with halo-model matter power
plt.subplot(3, 1, 3)
plt.semilogx(ks, Pks_lin/Pk_gg, color='black')
for (Pk, col, ls) in zip(Pks, cols, lss):
    plt.semilogx(ks, Pk/Pk_gg, color=col, ls=ls)
plt.xlabel(klab)
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_{uv}(k)\,/\,P_\mathrm{gg}^\mathrm{hm}(k)$')
plt.ylim((smin, smax))

# Finish
plt.tight_layout()
plt.show()

We can see that the central-central, central-satellite, satellite-central and central-central terms all contribute roughly equally at large scales (which is a coincidence for this particular HOD choice with this cosmology and redshift; note that twice the central-satellite term is plotted), then the satellite auto spectrum dominates at intermediate scales, while the central-satellite cross spectrum dominates at the smallest scales. Note that the central-central power is exactly a linearly biased version of the linear spectrum, with no one-halo term. This is because the shot-noise subtraction that is applied to spectra of discrete tracers completely cancels the one-halo term in this case (it would do the same for halo-halo spectra).